# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room Part #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.2.0](https://python.langchain.com/v0.2/docs/versions/v0_2/)
  4. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas) framework.
  

- 🤝 Breakout Room Part #2:
  1. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

> NOTE: Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage `GPT-3.5-Turbo` as the `critic_llm`!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room Part #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph 0.0.30 requires langchain-core<0.2.0,>=0.1.25, but you have langchain-core 0.2.9 which is incompatible.


We'll also get the "star of the show" today, which is Ragas!

In [2]:
!pip install -qU ragas

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arxiv 2.1.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
langgraph 0.0.30 requires langchain-core<0.2.0,>=0.1.25, but you have langchain-core 0.2.9 which is incompatible.


We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [3]:
!pip install -qU qdrant-client pymupdf pandas

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
wandb 0.17.0 requires protobuf!=4.21.0,<5,>=3.19.0; sys_platform != "linux", but you have protobuf 5.27.1 which is incompatible.
googleapis-common-protos 1.62.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
opentelemetry-proto 1.22.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.27.1 which is incompatible.


## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [4]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.2.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

documents = loader.load()

In [6]:
documents[0].metadata

{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'page': 0,
 'total_pages': 195,
 'format': 'PDF 1.3',
 'title': 'The Pmarca Blog Archives',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'Mac OS X 10.10 Quartz PDFContext',
 'creationDate': "D:20150110020418Z00'00'",
 'modDate': "D:20150110020418Z00'00'",
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [8]:
len(documents)

1864

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [9]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a QDrant VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [10]:
from langchain_community.vectorstores import Qdrant

qdrant_vector_store = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="PMarca Blogs",
)

####❓ Question #1:

List out a few of the techniques that Qdrant uses that make it performant.

> NOTE: Check the [documentation](https://qdrant.tech/documentation/overview/) for more information about QDrant!

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [11]:
retriever = qdrant_vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [12]:
retrieved_documents = retriever.invoke("What is a rule of thumb for selecting an industry to invest in?")

In [13]:
for doc in retrieved_documents:
  print(doc)

page_content='the existing order — and make sure that those forces of change\nhave a reasonable chance at succeeding.\nSecond rule of thumb:\nOnce you have picked an industry, get right to the center of it' metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 125, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '9a714252dc3b42e4a6c372a607d0ab2d', '_collection_name': 'PMarca Blogs'}
page_content='Third rule:\nIn a rapidly changing Held like technology, the best place to\nget experience when you’re starting out is in younger, high-\ngrowth companies.' metadata={'source': 'https://d1lamhf6l6yk6d

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [14]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [15]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [16]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [17]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

Let's test it out!

In [18]:
question = "What is a rule of thumb for selecting an industry to invest in?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Get right to the center of it.


In [19]:
question = "What did Pink Floyd have to say about how to proceed when investing in a new industry?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I don't know.
[Document(page_content='ask if you can call them again if things change.\nTrust me — they’d much rather be saying “yes” than “no” —\nthey need all the good investments they can get.\nSecond, consider the environment.', metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 15, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '858f561d03104acaacfcac0584acfb96', '_collection_name': 'PMarca Blogs'}), Document(page_content='watching carefully — if everyone agrees right up front that\nwhatever you are doing makes total sense, it probably isn’t a new\nand radical enough idea to justify a

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

## Task 4: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

In [20]:
loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

In [21]:
len(eval_documents)

624


> NOTE: 🛑 Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage GPT-3.5-Turbo as the critic_llm. If you're attempting to create a lot of samples please be aware of cost, as well as rate limits. 🛑

In [22]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
# critic_llm = ChatOpenAI(model="gpt-3.5-turbo") <--- If you don't have GPT-4 access, or to reduce cost/rate limiting issues.
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

testset = generator.generate_with_langchain_docs(eval_documents, 20, distributions, is_async = False)
testset.to_pandas()

/opt/homebrew/Caskroom/miniforge/base/envs/llmops-course/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Filename and doc_id are the same for all nodes.                     
Generating: 100%|██████████| 20/20 [00:46<00:00,  2.35s/it]


,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the definition of the caliber of a sta...,[Let’s start by deXning terms.\nThe caliber of...,The caliber of a startup team can be defined a...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,What is Los Angeles known for in terms of oppo...,[most interesting opportunity available — the ...,Los Angeles is known for opportunities and ind...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,What are the common layers of risk for a high-...,[as if it’s an onion. Just like you peel an on...,Here are some of the common layers of risk for...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,What impact does the presence of a billion peo...,[billion people online now. That is a very lar...,The presence of a billion people online has le...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,What is a key feature of Peter Hamilton's stor...,"[pandemics, nuclear terrorist attacks, governm...",Peter Hamilton's storytelling style in his spa...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,How can not keeping a schedule make a differen...,"[for me, but they probably won’t for you. Mayb...",nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,How does the tendency for quick elimination of...,[Four: Doubt-Avoidance Tendency\nThe brain of ...,The tendency for quick elimination of doubt in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,What is the importance of having a technical e...,[workforce in a high-impact way when you gradu...,Having a technical element in undergraduate de...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,What is the purpose of temporary subfolders in...,[you can reply to a lot of messages with “I’m ...,The purpose of temporary subfolders in email o...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,What are some potential unanticipated setbacks...,[Here’s why you shouldn’t do that:\nWhat are t...,You may have unanticipated setbacks within you...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

Let's look at the output and see what we can learn about it!

In [23]:
testset.test_data[0]

DataRow(question='What is the definition of the caliber of a startup team?', contexts=['Let’s start by deXning terms.\nThe caliber of a startup team\nteam can be deXned as the suitability of\nthe CEO, senior staW, engineers, and other key staW relative to\nthe opportunity in front of them.\nYou look at a startup and ask, will this team be able to optimally\nexecute against their opportunity? I focus on eWectiveness as\nopposed to experience, since the history of the tech industry is\nfull of highly successful startups that were staWed primarily by\npeople who had never “done it before”.\nThe quality of a startup’s pr\nproduct\noduct can be deXned as how impres-'], ground_truth='The caliber of a startup team can be defined as the suitability of the CEO, senior staff, engineers, and other key staff relative to the opportunity in front of them. It focuses on effectiveness rather than experience.', evolution_type='simple', metadata=[{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [24]:
test_df = testset.to_pandas()

In [25]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the definition of the caliber of a sta...,[Let’s start by deXning terms.\nThe caliber of...,The caliber of a startup team can be defined a...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,What is Los Angeles known for in terms of oppo...,[most interesting opportunity available — the ...,Los Angeles is known for opportunities and ind...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,What are the common layers of risk for a high-...,[as if it’s an onion. Just like you peel an on...,Here are some of the common layers of risk for...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,What impact does the presence of a billion peo...,[billion people online now. That is a very lar...,The presence of a billion people online has le...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,What is a key feature of Peter Hamilton's stor...,"[pandemics, nuclear terrorist attacks, governm...",Peter Hamilton's storytelling style in his spa...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,How can not keeping a schedule make a differen...,"[for me, but they probably won’t for you. Mayb...",nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,How does the tendency for quick elimination of...,[Four: Doubt-Avoidance Tendency\nThe brain of ...,The tendency for quick elimination of doubt in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,What is the importance of having a technical e...,[workforce in a high-impact way when you gradu...,Having a technical element in undergraduate de...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,What is the purpose of temporary subfolders in...,[you can reply to a lot of messages with “I’m ...,The purpose of temporary subfolders in email o...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,What are some potential unanticipated setbacks...,[Here’s why you shouldn’t do that:\nWhat are t...,You may have unanticipated setbacks within you...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


In [26]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [27]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [28]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [29]:
response_dataset[0]

{'question': 'What is the definition of the caliber of a startup team?',
 'answer': 'The caliber of a startup team can be defined as the suitability of the CEO, senior staff, engineers, and other key staff relative to the opportunity in front of them.',
 'contexts': ['Let’s start by deXning terms.\nThe caliber of a startup team\nteam can be deXned as the suitability of\nthe CEO, senior staW, engineers, and other key staW relative to\nthe opportunity in front of them.',
  '— team, product, and market.\nAt any given startup, the team will range from outstanding to\nremarkably Yawed; the product will range from a masterpiece',
  'course outright fail.\nSecond obvious fact: there is an incredibly wide divergence of\ncaliber and quality for the three core elements of each startup\n— team, product, and market.',
  'answer, in part because in the beginning of a startup, you know\na lot more about the team than you do the product, which hasn’t\nbeen built yet, or the market, which hasn’t been 

# 🤝 Breakout Room Part #2

## Task 1: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [30]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [31]:
results = evaluate(response_dataset, metrics)

Evaluating: 100%|██████████| 100/100 [00:28<00:00,  3.50it/s]


In [32]:
results

{'faithfulness': 0.5880, 'answer_relevancy': 0.9069, 'context_recall': 0.7300, 'context_precision': 0.8181, 'answer_correctness': 0.5190}

In [33]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What is the definition of the caliber of a sta...,The caliber of a startup team can be defined a...,[Let’s start by deXning terms.\nThe caliber of...,The caliber of a startup team can be defined a...,1.000000,0.979370,0.5,0.805556,0.847616
1,What is Los Angeles known for in terms of oppo...,Los Angeles is known for opportunities and ind...,[the city where all the action is happening.\n...,Los Angeles is known for opportunities and ind...,0.500000,1.000000,1.0,1.000000,1.000000
2,What are the common layers of risk for a high-...,"Founder risk, technology risk",[What are the layers of risk for a high-tech\n...,Here are some of the common layers of risk for...,1.000000,0.982414,0.0,0.833333,0.596498
3,What impact does the presence of a billion peo...,The presence of a billion people online is cau...,[billion people online now. That is a very lar...,The presence of a billion people online has le...,1.000000,0.947434,0.5,1.000000,0.688908
4,What is a key feature of Peter Hamilton's stor...,Large-scale military combat,[story of an enigmatic agent for the all-power...,Peter Hamilton's storytelling style in his spa...,NaN,0.836724,1.0,0.916667,0.192816
5,How can not keeping a schedule make a differen...,Not keeping a schedule can make a difference i...,"[Don’t keep a schedule\nHe’s crazy, you say!\n...",nan,1.000000,0.975101,0.0,0.500000,0.184283
6,How does the tendency for quick elimination of...,The tendency for quick elimination of doubt in...,[Four: Doubt-Avoidance Tendency\nThe brain of ...,The tendency for quick elimination of doubt in...,0.666667,0.998746,1.0,0.916667,0.695961
7,What is the importance of having a technical e...,The importance of having a technical element i...,[Which undergraduate degrees are useful in\nth...,Having a technical element in undergraduate de...,1.000000,1.000000,1.0,1.000000,0.495806
8,What is the purpose of temporary subfolders in...,The purpose of temporary subfolders in email o...,"[the normal course of your day.\nFourth, aside...",The purpose of temporary subfolders in email o...,0.500000,1.000000,1.0,0.333333,0.674064
9,What are some potential unanticipated setbacks...,Not raising enough money risks the survival of...,[Here’s why you shouldn’t do that:\nWhat are t...,You may have unanticipated setbacks within you...,1.000000,0.942212,0.0,1.000000,0.205057


## Task 2: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

> NOTE: MultiQueryRetriever is expanded on [here](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever) but for now, the implementation is not important to our lesson!

In [34]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [35]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [36]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [37]:
response = retrieval_chain.invoke({"input": "Who is Taylor Swift fueding with?"})

In [38]:
print(response["answer"])

I'm sorry, but based on the context provided, I cannot determine who Taylor Swift is feuding with.


In [39]:
response = retrieval_chain.invoke({"input": "Why are they fueding?"})

In [40]:
print(response["answer"])

The text does not provide any information about a feud or conflict between individuals or groups. It mainly discusses human tendencies towards dislike and hate, the history of war, factors contributing to success in startups, and the shift of consumer attention towards online media.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [41]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [42]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [43]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating: 100%|██████████| 100/100 [00:44<00:00,  2.23it/s]


In [44]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What is the definition of the caliber of a sta...,The caliber of a startup team can be defined a...,[Let’s start by deXning terms.\nThe caliber of...,The caliber of a startup team can be defined a...,1.000000,0.979370,0.500000,0.805556,0.847616
1,What is Los Angeles known for in terms of oppo...,Los Angeles is known for opportunities and ind...,[the city where all the action is happening.\n...,Los Angeles is known for opportunities and ind...,1.000000,1.000000,1.000000,1.000000,0.669579
2,What are the common layers of risk for a high-...,The common layers of risk for a high-tech star...,[What are the layers of risk for a high-tech\n...,Here are some of the common layers of risk for...,1.000000,0.994624,0.000000,0.691667,0.743744
3,What impact does the presence of a billion peo...,The presence of a billion people online has le...,"[Which makes total sense, amid the enormous ma...",The presence of a billion people online has le...,1.000000,0.989127,0.500000,1.000000,0.823114
4,What is a key feature of Peter Hamilton's stor...,A key feature of Peter Hamilton's storytelling...,[story of an enigmatic agent for the all-power...,Peter Hamilton's storytelling style in his spa...,1.000000,0.999999,0.666667,0.805556,0.742864
5,How can not keeping a schedule make a differen...,Not keeping a schedule can make a difference i...,"[Don’t keep a schedule\nHe’s crazy, you say!\n...",nan,0.500000,0.973598,0.500000,0.416667,0.181093
6,How does the tendency for quick elimination of...,The tendency for quick elimination of doubt in...,[do…\nSo pronounced is the tendency in man to ...,The tendency for quick elimination of doubt in...,1.000000,0.886399,1.000000,0.916667,0.406556
7,What is the importance of having a technical e...,Having a technical element in undergraduate de...,[Which undergraduate degrees are useful in\nth...,Having a technical element in undergraduate de...,0.833333,0.987819,1.000000,1.000000,0.610993
8,What is the purpose of temporary subfolders in...,The purpose of temporary subfolders in email o...,"[matter, right now.\nThose subfolders then get...",The purpose of temporary subfolders in email o...,1.000000,0.998859,1.000000,0.333333,0.496802
9,What are some potential unanticipated setbacks...,Some potential unanticipated setbacks that can...,[Here’s why you shouldn’t do that:\nWhat are t...,You may have unanticipated setbacks within you...,1.000000,1.000000,0.000000,1.000000,0.218906


## Task 3: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [45]:
results

{'faithfulness': 0.5880, 'answer_relevancy': 0.9069, 'context_recall': 0.7300, 'context_precision': 0.8181, 'answer_correctness': 0.5190}

And see how our advanced retrieval modified our chain!

In [46]:
advanced_retrieval_results

{'faithfulness': 0.7506, 'answer_relevancy': 0.9099, 'context_recall': 0.6633, 'context_precision': 0.7939, 'answer_correctness': 0.5752}

In [47]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.587963,0.750556,0.162593
1,answer_relevancy,0.906894,0.909929,0.003036
2,context_recall,0.730000,0.663333,-0.066667
3,context_precision,0.818056,0.793893,-0.024163
4,answer_correctness,0.519006,0.575248,0.056242


## Task 4: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [48]:
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [49]:
vector_store = Qdrant.from_documents(
    documents,
    new_embeddings,
    location=":memory:",
    collection_name="PMarca Blogs - TE3 - MQR",
)

In [50]:
new_retriever = vector_store.as_retriever()

In [51]:
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

In [52]:
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [53]:
answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [54]:
new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [55]:
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

Evaluating: 100%|██████████| 100/100 [00:48<00:00,  2.05it/s]


In [58]:
new_advanced_retrieval_results

{'faithfulness': 0.7688, 'answer_relevancy': 0.9567, 'context_recall': 0.6517, 'context_precision': 0.7326, 'answer_correctness': 0.5532}

In [59]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA + Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA + MQR'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'TE3 + MQR'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['ADA + MQR -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + MQR']
df_merged['Baseline -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + Baseline']

df_merged

,Metric,ADA + Baseline,ADA + MQR,TE3 + MQR,ADA + MQR -> TE3 + MQR,Baseline -> TE3 + MQR
0,faithfulness,0.587963,0.750556,0.768750,0.018194,0.180787
1,answer_relevancy,0.906894,0.909929,0.956709,0.046779,0.049815
2,context_recall,0.730000,0.663333,0.651667,-0.011667,-0.078333
3,context_precision,0.818056,0.793893,0.732643,-0.061250,-0.085413
4,answer_correctness,0.519006,0.575248,0.553232,-0.022016,0.034226


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

## BONUS ACTIVITY: Using a Better Generator

Now that we've seen how much more effective a better Retrieval pipeline is, let's look at what impact a better(?) Generator is!

Adapt the above `TE3 + MQR` pipeline to use `GPT-4o` and compare the results below!

In [ ]:
### YOUR CODE HERE